# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

1.0029314903022404

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [symbol,
        data[symbol]['price'],
         data[symbol]['stats']['year1ChangePercent'],
        "N/A"],
        index=my_columns), ignore_index=True)


In [8]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.82,0.809129,N/A
1,AAL,25.10,1.12585,N/A
2,AAP,187.61,1.13343,N/A
3,AAPL,124.65,0.974952,N/A
4,ABBV,106.71,0.545137,N/A
...,...,...,...,...
500,YUM,113.15,0.819779,N/A
501,ZBH,160.89,1.02966,N/A
502,ZBRA,470.01,1.52226,N/A
503,ZION,59.10,1.16719,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)

In [10]:
final_dataframe = final_dataframe[:50]

In [11]:
final_dataframe.reset_index(inplace=True)

In [12]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,96.97,7.401,N/A
1,179,FCX,35.71,5.80177,N/A
2,275,LB,62.47,5.28419,N/A
3,305,MGM,40.76,4.54946,N/A
4,175,FANG,77.26,4.09176,N/A
5,212,HAL,23.16,3.98336,N/A
6,314,MOS,34.85,3.88736,N/A
7,462,URI,330.53,3.67647,N/A
8,45,APTV,151.70,3.51113,N/A
9,148,DVN,22.85,3.4633,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [13]:
MAX_TRIES = 10
def get_portfolio_size():
    tries = 0
    portfolio_size = None
    while portfolio_size is None:
        if tries < MAX_TRIES:
            portfolio_value_string = input("Enter the value of your portfolio... ")
        #     portfolio_value_string = 10000000
            try:
                portfolio_size = float(portfolio_value_string)
                print(portfolio_size)
                return portfolio_size
            
            except ValueError:
                print("That's not a number! Enter a numerical value of your portfolio in $.")
                tries += 1
        else:
            print("Taking 0 as input (-;")
            return 0
    
    return portfolio_size
    

In [14]:
portfolio_size = get_portfolio_size()

Enter the value of your portfolio... 10000000
10000000.0


In [15]:
position_size = float(portfolio_size)/len(final_dataframe.index)

In [16]:
position_size

200000.0

In [17]:
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i,"Price"])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [18]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,96.97,7.401,2062
1,179,FCX,35.71,5.80177,5600
2,275,LB,62.47,5.28419,3201
3,305,MGM,40.76,4.54946,4906
4,175,FANG,77.26,4.09176,2588
5,212,HAL,23.16,3.98336,8635
6,314,MOS,34.85,3.88736,5738
7,462,URI,330.53,3.67647,605
8,45,APTV,151.70,3.51113,1318
9,148,DVN,22.85,3.4633,8752


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [89]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    
    "One-Year Price Return",
    "One-Year Return Percentile",
    
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    
    "One-Month Price Return",    
    "One-Month Return Percentile",
    
    "HQM Score"
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [90]:
convert_none = lambda x : 0 if x is None else x

In [91]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [92]:
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([    
                                            symbol,
                                            convert_none(data[symbol]["price"]),
                                            "N/A",
                                            
                                            convert_none(data[symbol]["stats"]["year1ChangePercent"]),
                                            "N/A",

                                            convert_none(data[symbol]["stats"]["month6ChangePercent"]),
                                            "N/A",

                                            convert_none(data[symbol]["stats"]["month3ChangePercent"]),
                                            "N/A",
                                            
                                            convert_none(data[symbol]["stats"]["month1ChangePercent"]),
                                            "N/A",
                                            
                                            "N/A"],
                                            
                                         index = hqm_columns),
            
                                    ignore_index = True)


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [93]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.96,N/A,0.804949,N/A,0.248214,N/A,0.030298,N/A,-0.040648,N/A,N/A
1,AAL,25.90,N/A,1.132362,N/A,0.902098,N/A,0.500919,N/A,0.410591,N/A,N/A
2,AAP,183.95,N/A,1.094929,N/A,0.207273,N/A,0.152993,N/A,0.100545,N/A,N/A
3,AAPL,121.83,N/A,0.984183,N/A,0.133201,N/A,-0.046952,N/A,-0.071432,N/A,N/A
4,ABBV,105.70,N/A,0.541691,N/A,0.184388,N/A,0.005147,N/A,-0.021849,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.71,N/A,0.858202,N/A,0.192043,N/A,-0.009033,N/A,0.028996,N/A,N/A
501,ZBH,165.36,N/A,1.022260,N/A,0.153020,N/A,0.049652,N/A,0.013798,N/A,N/A
502,ZBRA,463.68,N/A,1.495548,N/A,0.838594,N/A,0.247840,N/A,-0.034779,N/A,N/A
503,ZION,58.50,N/A,1.201254,N/A,0.876239,N/A,0.422729,N/A,0.146385,N/A,N/A


In [137]:
from scipy.stats import percentileofscore as score
time_periods = [
                "One-Year",
                "Six-Month",
                "Three-Month",
                "One-Month"
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f"{time_period} Price Return"
        percentile_col = f"{time_period} Return Percentile"
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],
                                                                         hqm_dataframe.loc[row, change_col])/100

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [138]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,101.00,198,7.379878,1,2.266822,1,1.751884,0.98,0.588960,1,99.9505
1,DISCA,79.03,253,2.961422,0.82,2.235273,0.98,1.792467,1,0.538601,0.98,99.2574
2,DISCK,65.69,304,2.763025,0.76,1.964007,0.96,1.638073,0.96,0.530185,0.96,98.9109
3,LB,61.08,327,5.304225,0.96,1.037286,0.76,0.579980,0.88,0.238180,0.76,98.1188
4,KSS,60.33,331,2.926523,0.78,1.590535,0.9,0.543026,0.8,0.247423,0.8,97.8713
5,GPS,31.90,626,2.935105,0.8,0.871284,0.64,0.529722,0.78,0.340869,0.88,97.2277
6,TPR,46.68,428,2.453080,0.64,1.614867,0.92,0.593435,0.9,0.199914,0.62,96.9802
7,FANG,78.74,254,4.058236,0.92,1.275691,0.82,0.654676,0.94,0.165085,0.54,96.9802
8,MRO,10.89,1836,2.182905,0.56,1.271903,0.8,0.594052,0.92,0.176927,0.56,95.7426
9,FITB,40.33,495,2.535459,0.68,0.854769,0.58,0.501497,0.76,0.180746,0.58,95.3465


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [139]:
from statistics import mean

In [140]:
# change_col = f"{time_period} Price Return"
# percentile_col = f"{time_period} Return Percentile"

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f"{time_period} Return Percentile"])
        
    hqm_dataframe.loc[row, "HQM Score"] = mean(momentum_percentiles)    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [141]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,101.00,198,7.379878,1,2.266822,1,1.751884,0.98,0.588960,1,0.995
1,DISCA,79.03,253,2.961422,0.82,2.235273,0.98,1.792467,1,0.538601,0.98,0.945
2,DISCK,65.69,304,2.763025,0.76,1.964007,0.96,1.638073,0.96,0.530185,0.96,0.91
3,LB,61.08,327,5.304225,0.96,1.037286,0.76,0.579980,0.88,0.238180,0.76,0.84
4,KSS,60.33,331,2.926523,0.78,1.590535,0.9,0.543026,0.8,0.247423,0.8,0.82
5,GPS,31.90,626,2.935105,0.8,0.871284,0.64,0.529722,0.78,0.340869,0.88,0.775
6,TPR,46.68,428,2.453080,0.64,1.614867,0.92,0.593435,0.9,0.199914,0.62,0.77
7,FANG,78.74,254,4.058236,0.92,1.275691,0.82,0.654676,0.94,0.165085,0.54,0.805
8,MRO,10.89,1836,2.182905,0.56,1.271903,0.8,0.594052,0.92,0.176927,0.56,0.71
9,FITB,40.33,495,2.535459,0.68,0.854769,0.58,0.501497,0.76,0.180746,0.58,0.65


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [142]:
hqm_dataframe.sort_values("HQM Score", ascending = False, inplace = True)

<ipython-input-142-39e286893995>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values("HQM Score", ascending = False, inplace = True)


In [143]:
hqm_dataframe = hqm_dataframe[:50]

In [144]:
hqm_dataframe.reset_index(inplace=True, drop=True)

In [145]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,101.00,198,7.379878,1,2.266822,1,1.751884,0.98,0.588960,1,0.995
1,DISCA,79.03,253,2.961422,0.82,2.235273,0.98,1.792467,1,0.538601,0.98,0.945
2,DISCK,65.69,304,2.763025,0.76,1.964007,0.96,1.638073,0.96,0.530185,0.96,0.91
3,LB,61.08,327,5.304225,0.96,1.037286,0.76,0.579980,0.88,0.238180,0.76,0.84
4,KSS,60.33,331,2.926523,0.78,1.590535,0.9,0.543026,0.8,0.247423,0.8,0.82
5,FANG,78.74,254,4.058236,0.92,1.275691,0.82,0.654676,0.94,0.165085,0.54,0.805
6,GPS,31.90,626,2.935105,0.8,0.871284,0.64,0.529722,0.78,0.340869,0.88,0.775
7,TPR,46.68,428,2.453080,0.64,1.614867,0.92,0.593435,0.9,0.199914,0.62,0.77
8,MRO,10.89,1836,2.182905,0.56,1.271903,0.8,0.594052,0.92,0.176927,0.56,0.71
9,EXPE,187.10,106,2.969831,0.84,0.947761,0.72,0.442560,0.58,0.156295,0.48,0.655


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [146]:
portfolio_size = get_portfolio_size()

Enter the value of your portfolio... 
That's not a number! Enter a numerical value of your portfolio in $.
Enter the value of your portfolio... 1000000
1000000.0


In [147]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

In [148]:
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i , "Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[i, "Price"])

In [149]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,101.00,198,7.379878,1,2.266822,1,1.751884,0.98,0.588960,1,0.995
1,DISCA,79.03,253,2.961422,0.82,2.235273,0.98,1.792467,1,0.538601,0.98,0.945
2,DISCK,65.69,304,2.763025,0.76,1.964007,0.96,1.638073,0.96,0.530185,0.96,0.91
3,LB,61.08,327,5.304225,0.96,1.037286,0.76,0.579980,0.88,0.238180,0.76,0.84
4,KSS,60.33,331,2.926523,0.78,1.590535,0.9,0.543026,0.8,0.247423,0.8,0.82
5,FANG,78.74,254,4.058236,0.92,1.275691,0.82,0.654676,0.94,0.165085,0.54,0.805
6,GPS,31.90,626,2.935105,0.8,0.871284,0.64,0.529722,0.78,0.340869,0.88,0.775
7,TPR,46.68,428,2.453080,0.64,1.614867,0.92,0.593435,0.9,0.199914,0.62,0.77
8,MRO,10.89,1836,2.182905,0.56,1.271903,0.8,0.594052,0.92,0.176927,0.56,0.71
9,EXPE,187.10,106,2.969831,0.84,0.947761,0.72,0.442560,0.58,0.156295,0.48,0.655


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [166]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine = "xlsxwriter")
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [167]:
background_color = '#ffffff'
font_color = '#0a0a23'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'00.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [168]:
column_formats = {
    "A": ["Ticker", string_template],
    "B": ["Price", dollar_template],
    "C": ["Number of Shares to Buy", percent_template],
    "D": ["One-Year Price Return", percent_template],
    "E": ["One-Year Return Percentile", percent_template],
    "F": ["Six-Month Price Return", percent_template],
    "G": ["Six-Month Return Percentile", percent_template],
    "H": ["Three-Month Price Return", percent_template],
    "I": ["Three-Month Return Percentile", percent_template],
    "J": ["One-Month Price Return", percent_template],    
    "K": ["One-Month Return Percentile", percent_template],
    "L": ["HQM Score", percent_template]
    
}

for column in columns_format.keys():
    writer.sheets["Momentum Strategy"].set_column(f"{column}:{column}", 25 , column_formats[column][1])
    writer.sheets["Momentum Strategy"].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [169]:
writer.save()